Device tree overlay

In [ ]:
from redpitaya.overlay.mercury import mercury as FPGA
overlay = FPGA()

Hardware identification

In [ ]:
regid = FPGA.hwid()

hex(regid.dna)

In [ ]:
del(regid)

Arbitrary signal generator

In [ ]:
import numpy as np

gen0 = FPGA.gen(0)

x = np.linspace(-np.pi, np.pi, gen0.N, endpoint=False)
wave = np.sin(x)

gen0.amplitude = 1.0
gen0.offset    = 0
# waveform should be loaded before frequency and phase are calculated
# this provides the correct buffer size
gen0.waveform  = wave
gen0.frequency = 10000
gen0.phase     = 0
gen0.control_mask = gen0.event_mask
gen0.start()
gen0.trigger()
gen0.enable = True

In [ ]:
gen0.show_regset()

Acquire (oscilloscope)

In [ ]:
osc0 = FPGA.osc(0, 1.0)

In [ ]:
# TODO: for now bypass input filter
osc0.filter_bypass = True

# trigger timing
N = FPGA.osc.N
osc0.regset.cfg_pre = N//4*1
osc0.regset.cfg_pst = N//4*3

# trigger level [V], edge and holdoff [periods]
osc0.level = [-0.1, +0.1]
osc0.edg = 'pos'
osc0.holdoff = 20

# control/trigger event mask
# itself is the source of all events
osc0.control_mask = osc0.event_mask
osc0.trigger_mask = osc0.event_mask

osc0.reset()
osc0.start()
#osc0.trigger()
while (osc0.status_run()): pass
#osc0.stop()

In [ ]:
osc0.show_regset()

In [ ]:
import matplotlib.pyplot as plt

# show entire buffer while ignoring write pointer position
data = osc0.data(ptr=0)
plt.plot(data)
plt.title('DATA');
plt.show()

# show entire buffer properly alligned with write pointer
data = osc0.data()
plt.plot(data)
plt.title('DATA');
plt.show()

# show only the part of the buffer requested by pre/post trigger timing
data = osc0.data(N)
plt.plot(data)
plt.title('DATA');
plt.show()


In [ ]:
data[0:4], data[-5:-1]

In [ ]:
del(gen0)
del(osc0)

In [ ]:
del(overlay)